# Exploratory Data Analysis

After completing the preprocessing steps, an Exploratory Data Analysis (EDA) will be performed to explore trends and key insights within the dataset. The EDA will involve:

- Using summary statistics to provide an overview of the data.
- Employing visualizations such as histograms, scatter plots, and box plots to highlight data distributions and relationships.
- Conducting correlation analysis to uncover possible links between various metrics.

This analysis will help identify important patterns and relationships in NFL statistics.

## Index<a id='toc0_'></a>
1. [Libraries](#toc1_)
2. [Data Exploration](#toc2_)
   1. [Team Performance and Success Across Seasons](#toc2_1_)
   2. [Offensive Strategy and Productivity](#toc2_2_)
         1. [Scoring Efficiency and Point Differential Analysis](#toc2_2_1_)
         2. [Passing vs. Rushing](#toc2_2_2_)
   3. [Turnover Battle & Its Influence on Wins](#toc2_3_)
   4. [Penalties and Discipline Impact on Success](#toc2_4_)

## 1. <a id='toc1_'></a>[Libraries](#toc0_)

In [385]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import requests
import os
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import folium
from folium.plugins import FastMarkerCluster
import geopandas as gpd
from branca.colormap import LinearColormap
from folium.plugins import HeatMap

import plotly.graph_objs as go
import plotly_express as px
import chart_studio.plotly as py
from plotly.offline import iplot, init_notebook_mode
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import display
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import mannwhitneyu
from scipy.stats import skew
from scipy.stats import spearmanr
from scipy import stats
import logging
logging.basicConfig(filename='logs.log', level=logging.INFO)

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression

Reading preprocessed Dataset:

In [386]:
NFL_stats = pd.read_csv('/Users/lucianocufari/Documents/Bootcamp_Data/programacion/material_curso/Python/NFL_data_analysis/clean_data/NFL_stats_df_cleaned.csv.gz')

## 2. <a id='toc2_'></a>[Data Exploration](#toc0_)

Exploring this NFL data allows for a range of interesting analysis, diving into key performance metrics, patterns of success, and the comparison between teams.

1. *Team Performance and Success Across Seasons.*

2. *Offensive Strategy and Productivity.*

3. *Turnover Battle and Its Influence on Wins.*

4. *Penalties and Discipline Impact on Success.*

### 2.1. <a id='toc2_1_'></a>[Team Performance and Success Across Seasons](#toc0_)

In [387]:
team_stats = NFL_stats.groupby('Team').agg(
    avg_win_loss_perc=('Win-Loss Percentage', 'mean')
).reset_index()

team_stats = team_stats.sort_values(by='avg_win_loss_perc', ascending=False)

fig = go.Figure(go.Bar(
    x=team_stats['avg_win_loss_perc'], 
    y=team_stats['Team'], 
    orientation='h',  
    marker=dict(color=team_stats['avg_win_loss_perc'], colorscale='bluered_r')
))

fig.update_layout(
    title='Average Win/Loss Percentage of NFL Teams Over the Last 21 Years',
    xaxis_title='Average Win/Loss Percentage',
    yaxis_title='',
    yaxis=dict(autorange="reversed"),
    height=1000
)

fig.show()


In [388]:
win_loss_pivot = NFL_stats.pivot_table(values='Win-Loss Percentage', index='Team', columns='Year')

z_values = win_loss_pivot.values

heatmap = go.Heatmap(
    z=z_values,  
    x=win_loss_pivot.columns, 
    y=win_loss_pivot.index, 
    colorscale='bluered_r', 
    colorbar=dict(title='Win/Loss Percentage')
)

layout = go.Layout(
    title='Win-Loss Percentage Heatmap',
    xaxis=dict(title='Year'),
    yaxis=dict(title='', automargin=True),
    height=800, 
    width=1000  
)

fig = go.Figure(data=[heatmap], layout=layout)

fig.show()


In [389]:
team_stats = NFL_stats.groupby('Team').agg(
    avg_points_diff=('Points Differential', 'mean')
).reset_index()

team_stats = team_stats.sort_values(by='avg_points_diff', ascending=True)

bar_chart = go.Bar(
    x=team_stats['avg_points_diff'],  
    y=team_stats['Team'],  
    orientation='h',  
    marker=dict(
        color=team_stats['avg_points_diff'],  
        colorscale='bluered_r'  
    )
)

layout = go.Layout(
    title='Average Points Differential of NFL Teams Over the Last 21 Years',
    xaxis=dict(title='Average Points Differential'),
    yaxis=dict(title='', automargin=True),  
    height=800,  
    width=1000   
)

fig = go.Figure(data=[bar_chart], layout=layout)

fig.show()


In [390]:
super_bowl_winners = NFL_stats[NFL_stats['SuperBowl Win'] > 0]

super_bowl_wins_by_team = super_bowl_winners.groupby('Team')['SuperBowl Win'].sum().reset_index()

super_bowl_wins_by_team = super_bowl_wins_by_team.sort_values(by='SuperBowl Win', ascending=False)

team_colors = ['#002244', '#E31837', '#0B2265', '#FFB612', '#241773', '#FB4F14', '#203731',
                '#002C5F', '#003594', '#D3BC8D', '#004C54', '#002244', '#D50A0A']

pie_chart = go.Pie(
    labels=super_bowl_wins_by_team['Team'],  
    values=super_bowl_wins_by_team['SuperBowl Win'],  
    hoverinfo='label+percent',  
    textinfo='value+label',
    textposition='inside',
    marker=dict(
        colors=team_colors,  
        line=dict(color='black', width=2)  
    )
)

layout = go.Layout(
    title='Super Bowl Winners of the Last 21 Years',
    height=700,
    width=900,
    showlegend=False
)

fig = go.Figure(data=[pie_chart], layout=layout)

fig.show()


In [391]:
primary_team_colors = {'Baltimore Ravens': '#241773', 'Cincinnati Bengals': '#FB4F14',
                         'Cleveland Browns': '#311D00', 'Pittsburgh Steelers': '#000000',
                         'Buffalo Bills': '#00338D', 'Miami Dolphins': '#008E97',
                         'New England Patriots': '#002244', 'New York Jets': '#125740',
                         'Houston Texans': '#03202F', 'Indianapolis Colts': '#002C5F',
                         'Jacksonville Jaguars': '#006778', 'Tennessee Titans': '#0C2340',
                         'Denver Broncos': '#002244', 'Kansas City Chiefs': '#E31837',
                         'Las Vegas Raiders': '#000000', 'Los Angeles Chargers': '#0080C6',
                         'Chicago Bears': '#0B162A', 'Detroit Lions': '#0076B6',
                         'Green Bay Packers': '#203731', 'Minnesota Vikings': '#4F2683',
                         'Dallas Cowboys': '#003594', 'New York Giants': '#0B2265',
                         'Philadelphia Eagles': '#004C54', 'Washington Commanders': '#5A1414',
                         'Atlanta Falcons': '#A71930', 'Carolina Panthers': '#0085CA',
                         'New Orleans Saints': '#000000', 'Tampa Bay Buccaneers': '#D50A0A',
                         'Arizona Cardinals': '#97233F', 'Los Angeles Rams': '#003594',
                         'San Francisco 49ers': '#AA0000', 'Seattle Seahawks': '#002244'
                         }

In [392]:
team_dropdown = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Select Team:"
)

def update_plot(selected_team):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]
    
    fig = px.line(filtered_data, x='Year', y='Win-Loss Percentage', color='Team',
                  title=f"Win-Loss Percentage Trend for the {selected_team}",
                  color_discrete_map={selected_team: primary_team_colors.get(selected_team, '#000000')})
    
    fig.update_yaxes(range=[0, 1])
    fig.update_layout(showlegend=False)
    fig.show()
    
    display(team_dropdown)

team_dropdown.observe(lambda change: update_plot(change['new']), names='value')

display(team_dropdown)

Dropdown(description='Select Team:', index=1, options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bill…

In [393]:
home_road_color_map = {
    'New England Patriots_Home': '#002244', 'New England Patriots_Road': '#C60C30',
    'Miami Dolphins_Home': '#008E97', 'Miami Dolphins_Road': '#FC4C02',
    'Buffalo Bills_Home': '#00338D', 'Buffalo Bills_Road': '#C60C30',
    'New York Jets_Home': '#125740', 'New York Jets_Road': '#000000',
    'Baltimore Ravens_Home': '#241773', 'Baltimore Ravens_Road': '#000000',
    'Cincinnati Bengals_Home': '#FB4F14', 'Cincinnati Bengals_Road': '#000000',
    'Cleveland Browns_Home': '#311D00', 'Cleveland Browns_Road': '#FF3C00',
    'Pittsburgh Steelers_Home': '#F5AB00', 'Pittsburgh Steelers_Road': '#000000',
    'Houston Texans_Home': '#03202F', 'Houston Texans_Road': '#A71930',
    'Indianapolis Colts_Home': '#002C5F', 'Indianapolis Colts_Road': '#889296',
    'Jacksonville Jaguars_Home': '#006778', 'Jacksonville Jaguars_Road': '#000000',
    'Tennessee Titans_Home': '#0C2340', 'Tennessee Titans_Road': '#4B92DB',
    'Denver Broncos_Home': '#002244', 'Denver Broncos_Road': '#FB4F14',
    'Kansas City Chiefs_Home': '#E31837', 'Kansas City Chiefs_Road': '#F5AB00',
    'Las Vegas Raiders_Home': '#000000', 'Las Vegas Raiders_Road': '#737D82',
    'Los Angeles Chargers_Home': '#0080C6', 'Los Angeles Chargers_Road': '#FFC20E',
    'Chicago Bears_Home': '#0B162A', 'Chicago Bears_Road': '#C83803',
    'Detroit Lions_Home': '#0076B6', 'Detroit Lions_Road': '#869098',
    'Green Bay Packers_Home': '#203731', 'Green Bay Packers_Road': '#F5AB00',
    'Minnesota Vikings_Home': '#4F2683', 'Minnesota Vikings_Road': '#F5AB00',
    'Dallas Cowboys_Home': '#003594', 'Dallas Cowboys_Road': '#718084',
    'New York Giants_Home': '#0B2265', 'New York Giants_Road': '#A71930',
    'Philadelphia Eagles_Home': '#004C54', 'Philadelphia Eagles_Road': '#718084',
    'Washington Commanders_Home': '#5A1414', 'Washington Commanders_Road': '#F5AB00',
    'Atlanta Falcons_Home': '#A71930', 'Atlanta Falcons_Road': '#000000',
    'Carolina Panthers_Home': '#0085CA', 'Carolina Panthers_Road': '#000000',
    'New Orleans Saints_Home': '#C6A86C', 'New Orleans Saints_Road': '#000000',
    'Tampa Bay Buccaneers_Home': '#D50A0A', 'Tampa Bay Buccaneers_Road': '#FF7900',
    'Arizona Cardinals_Home': '#97233F', 'Arizona Cardinals_Road': '#000000',
    'Los Angeles Rams_Home': '#003594', 'Los Angeles Rams_Road': '#FFA300',
    'San Francisco 49ers_Home': '#AA0000', 'San Francisco 49ers_Road': '#B3995D',
    'Seattle Seahawks_Home': '#002244', 'Seattle Seahawks_Road': '#69BE28'
}

home_road_df = NFL_stats.melt(id_vars=['Team', 'Year'], value_vars=['Home', 'Road'], var_name='Location', value_name='Record')

home_road_df['team_location'] = home_road_df['Team'] + "_" + home_road_df['Location']

team_dropdown_1 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Select Team:"
)

def update_plot_1(selected_team):
    clear_output(wait=True)
    filtered_data = home_road_df[home_road_df['Team'] == selected_team]
    
    fig = px.bar(filtered_data, x='Record', y='Location', color='team_location', barmode='group', orientation='h',
                  title=f"Home vs Road Performance for the {selected_team}",
                  color_discrete_map=home_road_color_map)
    
    fig.update_layout(xaxis_title='Sum of the Record Percentage', yaxis_title='',
                      yaxis=dict(showticklabels=False),
                      bargap=0.2,
                      bargroupgap=0,
                      height=300,
                      legend_title_text='Venue')
    
    new_legend_names = {
        f"{selected_team}_Home": "Home",
        f"{selected_team}_Road": "Road"
    }
    fig.for_each_trace(lambda t: t.update(name=new_legend_names.get(t.name, t.name)))
    
    fig.show()
    
    display(team_dropdown_1)

team_dropdown_1.observe(lambda change: update_plot_1(change['new']), names='value')

display(team_dropdown_1)

Dropdown(description='Select Team:', index=1, options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bill…

In [394]:
conference_color_map = {
    'New England Patriots_Division': '#002244', 'New England Patriots_Conference': '#C60C30', 'New England Patriots_Non-Conference': '#7B868E',
    'Miami Dolphins_Division': '#00BECC', 'Miami Dolphins_Conference': '#FC4C02', 'Miami Dolphins_Non-Conference': '#005778',
    'Buffalo Bills_Division': '#005AF5', 'Buffalo Bills_Conference': '#C60C30', 'Buffalo Bills_Non-Conference': '#001E52',
    'New York Jets_Division': '#26BA89', 'New York Jets_Conference': '#000000', 'New York Jets_Non-Conference': '#0A3325',
    'Baltimore Ravens_Division': '#241773', 'Baltimore Ravens_Conference': '#000000', 'Baltimore Ravens_Non-Conference': '#9E7C0C',
    'Cincinnati Bengals_Division': '#FB4F14', 'Cincinnati Bengals_Conference': '#000000', 'Cincinnati Bengals_Non-Conference': '#B52F03',
    'Cleveland Browns_Division': '#311D00', 'Cleveland Browns_Conference': '#FF3C00', 'Cleveland Browns_Non-Conference': '#A36200',
    'Pittsburgh Steelers_Division': '#FFB612', 'Pittsburgh Steelers_Conference': '#000000', 'Pittsburgh Steelers_Non-Conference': '#737D82',
    'Houston Texans_Division': '#03202F', 'Houston Texans_Conference': '#A71930', 'Houston Texans_Non-Conference': '#0A6A99',
    'Indianapolis Colts_Division': '#002C5F', 'Indianapolis Colts_Conference': '#889296', 'Indianapolis Colts_Non-Conference': '#005FCC',
    'Jacksonville Jaguars_Division': '#006778', 'Jacksonville Jaguars_Conference': '#000000', 'Jacksonville Jaguars_Non-Conference': '#9F792C',
    'Tennessee Titans_Division': '#0C2340', 'Tennessee Titans_Conference': '#4B92DB', 'Tennessee Titans_Non-Conference': '#C8102E',
    'Denver Broncos_Division': '#002244', 'Denver Broncos_Conference': '#FB4F14', 'Denver Broncos_Non-Conference': '#005CB8',
    'Kansas City Chiefs_Division': '#E31837', 'Kansas City Chiefs_Conference': '#FFB81C', 'Kansas City Chiefs_Non-Conference': '#941023',
    'Las Vegas Raiders_Division': '#000000', 'Las Vegas Raiders_Conference': '#737D82', 'Las Vegas Raiders_Non-Conference': '#43494C',
    'Los Angeles Chargers_Division': '#0080C6', 'Los Angeles Chargers_Conference': '#FFC20E', 'Los Angeles Chargers_Non-Conference': '#004266',
    'Chicago Bears_Division': '#0B162A', 'Chicago Bears_Conference': '#C83803', 'Chicago Bears_Non-Conference': '#224381',
    'Detroit Lions_Division': '#0076B6', 'Detroit Lions_Conference': '#869098', 'Detroit Lions_Non-Conference': '#000000',
    'Green Bay Packers_Division': '#203731', 'Green Bay Packers_Conference': '#FFB612', 'Green Bay Packers_Non-Conference': '#4B8172',
    'Minnesota Vikings_Division': '#4F2683', 'Minnesota Vikings_Conference': '#FFB612', 'Minnesota Vikings_Non-Conference': '#26123F',
    'Dallas Cowboys_Division': '#003594', 'Dallas Cowboys_Conference': '#718084', 'Dallas Cowboys_Non-Conference': '#00153D',
    'New York Giants_Division': '#0B2265', 'New York Giants_Conference': '#A71930', 'New York Giants_Non-Conference': '#737D82',
    'Philadelphia Eagles_Division': '#004C54', 'Philadelphia Eagles_Conference': '#718084', 'Philadelphia Eagles_Non-Conference': '#000000',
    'Washington Commanders_Division': '#5A1414', 'Washington Commanders_Conference': '#FFB612', 'Washington Commanders_Non-Conference': '#210707',
    'Atlanta Falcons_Division': '#A71930', 'Atlanta Falcons_Conference': '#000000', 'Atlanta Falcons_Non-Conference': '#737D82',
    'Carolina Panthers_Division': '#0085CA', 'Carolina Panthers_Conference': '#000000', 'Carolina Panthers_Non-Conference': '#737D82',
    'New Orleans Saints_Division': '#C6A86C', 'New Orleans Saints_Conference': '#000000', 'New Orleans Saints_Non-Conference': '#937539',
    'Tampa Bay Buccaneers_Division': '#D50A0A', 'Tampa Bay Buccaneers_Conference': '#FF7900', 'Tampa Bay Buccaneers_Non-Conference': '#737D82',
    'Arizona Cardinals_Division': '#97233F', 'Arizona Cardinals_Conference': '#000000', 'Arizona Cardinals_Non-Conference': '#521421',
    'Los Angeles Rams_Division': '#003594', 'Los Angeles Rams_Conference': '#FFA300', 'Los Angeles Rams_Non-Conference': '#00153D',
    'San Francisco 49ers_Division': '#AA0000', 'San Francisco 49ers_Conference': '#B3995D', 'San Francisco 49ers_Non-Conference': '#000000',
    'Seattle Seahawks_Division': '#002244', 'Seattle Seahawks_Conference': '#69BE28', 'Seattle Seahawks_Non-Conference': '#737D82'
}

In [395]:
comp_df = NFL_stats.melt(id_vars=['Team', 'Year'], value_vars=['Division', 'Conference', 'Non-Conference'], 
                           var_name='Competition Type', value_name='Record')

comp_df['team_competition'] = comp_df['Team'] + '_' + comp_df['Competition Type']

team_dropdown_2 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Select Team:"
)

def update_plot_2(selected_team):
    clear_output(wait=True)
    filtered_data = comp_df[comp_df['Team'] == selected_team]

    fig = px.bar(filtered_data, x='Record', y='Team', color='team_competition', 
             barmode='group', orientation='h', 
             title=f'Divisional, Conference, and Non-Conference Performance for the {selected_team}', 
             color_discrete_map=conference_color_map)
    
    fig.update_layout(xaxis_title='Sum of the Record Percentage', yaxis_title='',
                      yaxis=dict(showticklabels=False),
                      bargap=0.2,
                      bargroupgap=0,
                      height=300,
                      legend_title_text='Rival')
    
    new_legend_names = {
        f"{selected_team}_Division": "Divisional",
        f"{selected_team}_Conference": "Conference",
        f"{selected_team}_Non-Conference": "Non-Conference"
    }
    fig.for_each_trace(lambda t: t.update(name=new_legend_names.get(t.name, t.name)))

    fig.show()
    
    display(team_dropdown_2)

team_dropdown_2.observe(lambda change: update_plot_2(change['new']), names='value')

display(team_dropdown_2)

Dropdown(description='Select Team:', index=1, options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bill…

In [396]:
team_dropdown_3 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Select Team:"
)

def update_plot_3(selected_team):
    clear_output(wait=True)
    filtered_data = comp_df[comp_df['Team'] == selected_team]

    fig = px.box(filtered_data, x='Team', y='Record', color='team_competition', 
                 title=f'Spread of Divisional, Conference, and Non-Conference Performance for the {selected_team}', 
                 color_discrete_map=conference_color_map)
    
    fig.update_yaxes(range=[-0.05, 1.05])
    
    fig.update_layout(xaxis=dict(showticklabels=False),
                      xaxis_title='', 
                      yaxis_title='Spread of Performance',
                      legend_title_text='Rival')
    
    new_legend_names = {
        f"{selected_team}_Division": "Divisional",
        f"{selected_team}_Conference": "Conference",
        f"{selected_team}_Non-Conference": "Non-Conference"
    }
    fig.for_each_trace(lambda t: t.update(name=new_legend_names.get(t.name, t.name)))
    
    fig.show()

    display(team_dropdown_3)

team_dropdown_3.observe(lambda change: update_plot_3(change['new']), names='value')

display(team_dropdown_3)

Dropdown(description='Select Team:', index=1, options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bill…

In [397]:
correlation_df = NFL_stats[['Streak', 'Last 5', 'SuperBowl Win']]

correlation_matrix = correlation_df.corr()

fig = px.imshow(correlation_matrix, 
                text_auto=True, 
                color_continuous_scale='bluered_r', 
                zmin=-1, zmax=1, 
                labels={'color': 'Correlation'},
                title='Correlation Between Streaks, End-of-Season Performance, and Championship Success')

fig.show()


In [398]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Streak']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Streak']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in streaks between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in streaks between teams that won and teams that did not win the Super Bowl.')


Mann-Whitney U Test: Stat = 9308.5, P-Value = 0.003924779509261738, There is a significant difference in streaks between teams that won and teams that did not win the Super Bowl.


In [399]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Last 5']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Last 5']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in End-Of-Season performance between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in End-Of-Season performance between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 9744.5, P-Value = 0.0007106729653572074, There is a significant difference in End-Of-Season performance between teams that won and teams that did not win the Super Bowl.


There is not a strong linear relationship between streaks/end-of-season-performance and Super Bowl wins. That is, having a better streak/end-of-season-performance is not directly associated in a linear fashion with winning the Super Bowl. Still, there is a difference in the distribution of streaks and end-of-season performance. Super Bowl winning teams tend to have better streaks and finish the regular season stronger, but the relationship is not strong enough to be reflected in a linear correlation.

### 2.2. <a id='toc2_2_'></a>[Offensive Strategy and Productivity](#toc0_)

#### 2.2.1. <a id='toc2_2_1_'></a>[Scoring Efficiency and Point Differential Analysis](#toc0_)

In [400]:
fig = px.scatter(NFL_stats, x='Points', y='Points Allowed', color='Win-Loss Percentage', 
                 size=abs(NFL_stats['Points Differential']),  
                 hover_data=['Team', 'Year'], 
                 title="Points Scored vs Points Allowed and Team Success",
                 labels={'Points': 'Points Scored'},
                 color_continuous_scale='Bluered_r')

fig.update_layout(coloraxis_colorbar=dict(title="Win-Loss %"))
fig.show()

The size of the points represents the point differential, and the color represents the win-loss percentage. You can look for teams with high point differentials and see if they typically have a high win percentage.

In [401]:
metrics_dropdown_3_5 = widgets.Dropdown(
    options=["Points Scored & Allowed", "Scoring Percentage"],
    description="Metric:"
)

NFL_stats['Points Differential Ratio'] = NFL_stats['Points'] / NFL_stats['Points Allowed']

metric_map = {
    "Points Scored & Allowed": "Points Differential Ratio",
    "Scoring Percentage": "Scoring Percentage" 
}

def update_plot_3_5(selected_metric):
    clear_output(wait=True)

    metric = metric_map[selected_metric]
    
    fig = px.bar(NFL_stats.sort_values(metric, ascending=False), x='Team', y=metric, color=metric, 
                 title=f"Scoring Efficiency: {metric}", color_continuous_scale='Bluered_r')
    
    fig.update_layout(yaxis_title=f'Sum of {metric}',
                  xaxis_title='',
                  height=700)

    if selected_metric == "Points Scored & Allowed":

        fig.update_layout(coloraxis_colorbar=dict(title="Ratio"))

    elif selected_metric == "Scoring Percentage":

        fig.update_layout(coloraxis_colorbar=dict(title="Percentage"))

    fig.show()

    display(metrics_dropdown_3_5)

metrics_dropdown_3_5.observe(lambda change: update_plot_3_5(change['new']), names='value')

display(metrics_dropdown_3_5)

Dropdown(description='Metric:', index=1, options=('Points Scored & Allowed', 'Scoring Percentage'), value='Sco…

In [402]:
metrics_dropdown_3_75 = widgets.Dropdown(
    options=["Points Scored & Allowed", "Scoring Percentage"],
    description="Metric:"
)

points_differential_ratio = NFL_stats.groupby('Team', as_index=False)['Points Differential Ratio'].sum()
points_differential_ratio = points_differential_ratio.sort_values('Points Differential Ratio', ascending=False)

scoring_percentage = NFL_stats.groupby('Team', as_index=False)['Scoring Percentage'].sum()
scoring_percentage = scoring_percentage.sort_values('Scoring Percentage', ascending=False)

metric_map_1 = {
    "Points Scored & Allowed": {
        'data': points_differential_ratio.rename(columns={'Points Differential Ratio': 'efficiency'}),
        'title': 'Points Differential Ratio',
        'yaxis_title': 'Sum of Points Differential Ratio'
    },
    "Scoring Percentage": {
        'data': scoring_percentage.rename(columns={'Scoring Percentage': 'efficiency'}),
        'title': 'Scoring Percentage',
        'yaxis_title': 'Sum of Scoring Percentage'
    }
}

def update_plot_3_75(selected_metric):
    clear_output(wait=True)

    metric_info = metric_map_1[selected_metric]
    metric_data = metric_info['data']
    yaxis_title = metric_info['yaxis_title']
    title = metric_info['title']

    fig = px.bar(metric_data, x='Team', y='efficiency', 
                 color='efficiency', title=f"Total Teams' Efficiency: {title}",
                 color_continuous_scale='Bluered_r')

    fig.update_layout(yaxis_title=yaxis_title,
                      xaxis_title='',
                      height=700,
                      coloraxis_showscale=False) 

    fig.show()

    display(metrics_dropdown_3_75)

metrics_dropdown_3_75.observe(lambda change: update_plot_3_75(change['new']), names='value')

display(metrics_dropdown_3_75)

Dropdown(description='Metric:', index=1, options=('Points Scored & Allowed', 'Scoring Percentage'), value='Sco…

In [403]:
points_color_map = {
    'New England Patriots_Points': '#002244', 'New England Patriots_Points Allowed': '#C60C30',
    'Miami Dolphins_Points': '#008E97', 'Miami Dolphins_Points Allowed': '#FC4C02',
    'Buffalo Bills_Points': '#00338D', 'Buffalo Bills_Points Allowed': '#C60C30',
    'New York Jets_Points': '#1F9870', 'New York Jets_Points Allowed': '#000000',
    'Baltimore Ravens_Points': '#3925BB', 'Baltimore Ravens_Points Allowed': '#000000',
    'Cincinnati Bengals_Points': '#FB4F14', 'Cincinnati Bengals_Points Allowed': '#000000',
    'Cleveland Browns_Points': '#311D00', 'Cleveland Browns_Points Allowed': '#FF3C00',
    'Pittsburgh Steelers_Points': '#F5AB00', 'Pittsburgh Steelers_Points Allowed': '#000000',
    'Houston Texans_Points': '#03202F', 'Houston Texans_Points Allowed': '#D5203E',
    'Indianapolis Colts_Points': '#002C5F', 'Indianapolis Colts_Points Allowed': '#889296',
    'Jacksonville Jaguars_Points': '#00ADCC', 'Jacksonville Jaguars_Points Allowed': '#000000',
    'Tennessee Titans_Points': '#0C2340', 'Tennessee Titans_Points Allowed': '#4B92DB',
    'Denver Broncos_Points': '#002244', 'Denver Broncos_Points Allowed': '#FB4F14',
    'Kansas City Chiefs_Points': '#E31837', 'Kansas City Chiefs_Points Allowed': '#F5AB00',
    'Las Vegas Raiders_Points': '#000000', 'Las Vegas Raiders_Points Allowed': '#9EA5A9',
    'Los Angeles Chargers_Points': '#0080C6', 'Los Angeles Chargers_Points Allowed': '#FFC20E',
    'Chicago Bears_Points': '#0B162A', 'Chicago Bears_Points Allowed': '#C83803',
    'Detroit Lions_Points': '#0076B6', 'Detroit Lions_Points Allowed': '#869098',
    'Green Bay Packers_Points': '#203731', 'Green Bay Packers_Points Allowed': '#F5AB00',
    'Minnesota Vikings_Points': '#4F2683', 'Minnesota Vikings_Points Allowed': '#F5AB00',
    'Dallas Cowboys_Points': '#003594', 'Dallas Cowboys_Points Allowed': '#919EA1',
    'New York Giants_Points': '#0B2265', 'New York Giants_Points Allowed': '#D5203E',
    'Philadelphia Eagles_Points': '#00373D', 'Philadelphia Eagles_Points Allowed': '#919EA1',
    'Washington Commanders_Points': '#5A1414', 'Washington Commanders_Points Allowed': '#F5AB00',
    'Atlanta Falcons_Points': '#D5203E', 'Atlanta Falcons_Points Allowed': '#000000',
    'Carolina Panthers_Points': '#0085CA', 'Carolina Panthers_Points Allowed': '#000000',
    'New Orleans Saints_Points': '#C6A86C', 'New Orleans Saints_Points Allowed': '#000000',
    'Tampa Bay Buccaneers_Points': '#AF0808', 'Tampa Bay Buccaneers_Points Allowed': '#FF9233',
    'Arizona Cardinals_Points': '#C62F52', 'Arizona Cardinals_Points Allowed': '#000000',
    'Los Angeles Rams_Points': '#003594', 'Los Angeles Rams_Points Allowed': '#FFA300',
    'San Francisco 49ers_Points': '#8F0000', 'San Francisco 49ers_Points Allowed': '#C4B082',
    'Seattle Seahawks_Points': '#002244', 'Seattle Seahawks_Points Allowed': '#69BE28'
}

In [404]:
team_dropdown_4 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metrics_dropdown_4 = widgets.Dropdown(
    options=["Points", "Margin of Victory"],
    description="Metric:"
)

def update_plot_4(selected_team, selected_metrics):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]

    fig = go.Figure()

    if selected_metrics == "Points":
        points_color = points_color_map.get(f"{selected_team}_Points")
        points_allowed_color = points_color_map.get(f"{selected_team}_Points Allowed") 

        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Points'], 
                                 mode='lines', name='Points Scored',
                                 line=dict(color=points_color)))
        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Points Allowed'], 
                                 mode='lines', name='Points Allowed',
                                 line=dict(color=points_allowed_color)))

        fig.update_layout(title=f"Points Scored vs Points Allowed for the {selected_team}",
                          xaxis_title="Year",
                          yaxis_title="Points",
                          legend_title="")
        
        fig.update_yaxes(range=[150, 650])

    elif selected_metrics == "Margin of Victory":
        
        margin_color = primary_team_colors.get(selected_team)

        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Margin of Victory'], 
                                 mode='lines', name='Margin of Victory',
                                 line=dict(color=margin_color)))

        fig.update_layout(title=f"Margin of Victory Over Time for the {selected_team}",
                          xaxis_title="Year",
                          yaxis_title="Margin of Victory",
                          legend_title="")
        
        fig.update_yaxes(range=[-20, 25])

    fig.show()

    display(team_dropdown_4)
    display(metrics_dropdown_4)

def on_dropdown_change(change):
    update_plot_4(team_dropdown_4.value, metrics_dropdown_4.value)

team_dropdown_4.observe(on_dropdown_change, names='value')
metrics_dropdown_4.observe(on_dropdown_change, names='value')

display(team_dropdown_4)
display(metrics_dropdown_4)

Dropdown(description='Team:', index=1, options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'Ne…

Dropdown(description='Metric:', index=1, options=('Points', 'Margin of Victory'), value='Margin of Victory')

In [405]:
team_dropdown_16 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metrics_dropdown_16 = widgets.Dropdown(
    options=["Points Scored & Success", "Points Allowed & Success"],
    description="Metric:"
)

def update_plot_16(selected_team, selected_metric):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]

    fig = go.Figure()

    if selected_metric == "Points Scored & Success":

        hover_text = [
            f"Year: {year}" 
            for year in zip(filtered_data['Year'])
        ]

        team_color = primary_team_colors.get(selected_team)

        fig.add_trace(go.Scatter(x=filtered_data['Win-Loss Percentage'], y=filtered_data['Points'],
                                  mode='markers', hovertext=hover_text,
                                  marker=dict(color=team_color,
                                              size=15)))
        
        fig.update_layout(title=f"Points Scored vs Win-Loss Percentage for the {selected_team}",
                          xaxis_title="Win-Loss Percentage",
                          yaxis_title="Points Scored")

    elif selected_metric == "Points Allowed & Success":

        hover_text = [
            f"Year: {year}" 
            for year in zip(filtered_data['Year'])
        ]

        team_color = primary_team_colors.get(selected_team)

        fig.add_trace(go.Scatter(x=filtered_data['Win-Loss Percentage'], y=filtered_data['Points Allowed'],
                                  mode='markers', hovertext=hover_text,
                                  marker=dict(color=team_color,
                                              size=15)))
        
        fig.update_layout(title=f"Points Allowed vs Win-Loss Percentage for the {selected_team}",
                          xaxis_title="Win-Loss Percentage",
                          yaxis_title="Points Allowed")
    
    fig.show()
    
    display(team_dropdown_16)
    display(metrics_dropdown_16)

def on_dropdown_change(change):
    update_plot_16(team_dropdown_16.value, metrics_dropdown_16.value)

team_dropdown_16.observe(on_dropdown_change, names='value')
metrics_dropdown_16.observe(on_dropdown_change, names='value')

display(team_dropdown_16)
display(metrics_dropdown_16)

Dropdown(description='Team:', index=2, options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'Ne…

Dropdown(description='Metric:', index=1, options=('Points Scored & Success', 'Points Allowed & Success'), valu…

In [406]:
real_primary_team_colors = {'Baltimore Ravens': '#241773', 'Cincinnati Bengals': '#FB4F14',
                         'Cleveland Browns': '#311D00', 'Pittsburgh Steelers': '#FFB612',
                         'Buffalo Bills': '#00338D', 'Miami Dolphins': '#008E97',
                         'New England Patriots': '#002244', 'New York Jets': '#125740',
                         'Houston Texans': '#03202F', 'Indianapolis Colts': '#002C5F',
                         'Jacksonville Jaguars': '#006778', 'Tennessee Titans': '#0C2340',
                         'Denver Broncos': '#FB4F14', 'Kansas City Chiefs': '#E31837',
                         'Las Vegas Raiders': '#000000', 'Los Angeles Chargers': '#0080C6',
                         'Chicago Bears': '#0B162A', 'Detroit Lions': '#0076B6',
                         'Green Bay Packers': '#203731', 'Minnesota Vikings': '#4F2683',
                         'Dallas Cowboys': '#003594', 'New York Giants': '#0B2265',
                         'Philadelphia Eagles': '#004C54', 'Washington Commanders': '#5A1414',
                         'Atlanta Falcons': '#A71930', 'Carolina Panthers': '#0085CA',
                         'New Orleans Saints': '#D3BC8D', 'Tampa Bay Buccaneers': '#D50A0A',
                         'Arizona Cardinals': '#97233F', 'Los Angeles Rams': '#003594',
                         'San Francisco 49ers': '#AA0000', 'Seattle Seahawks': '#002244'
                         }

In [407]:
year_dropdown_5 = widgets.Dropdown(
    options=NFL_stats['Year'].unique(), 
    description="Year:"
)

metrics_dropdown_5 = widgets.Dropdown(
    options=["Expected Points", "Scoring Percentage"],
    description="Metric:"
)

def update_plot_5(selected_year, selected_metric):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Year'] == selected_year]

    fig = go.Figure()

    if selected_metric == "Expected Points":

     for team in filtered_data['Team'].unique():
         team_data = filtered_data[filtered_data['Team'] == team]
         team_color = real_primary_team_colors.get(team)
         team_record = team_data['Win-Loss Percentage'].values[0]
        
         fig.add_trace(go.Scatter(x=team_data['Expected Points Total'], y=team_data['Points'],
                                  mode='markers', name=team,
                                  marker=dict(size=team_data['Win-Loss Percentage'] * 100,
                                              color=team_color),
                                  hovertext=f"{team}' record: {team_record}"))

     fig.update_layout(title=f"Expected Points vs Actual Points for Teams in {selected_year}",
                       xaxis_title="Expected Points",
                       yaxis_title="Actual Points",
                       legend_title="Team",
                       height=600)
     
     fig.update_yaxes(range=[150, 687])
     fig.update_xaxes(range=[-399, 350])
     
    elif selected_metric == "Scoring Percentage":
       
       for team in filtered_data['Team'].unique():
         team_data = filtered_data[filtered_data['Team'] == team]
         team_color = real_primary_team_colors.get(team)
         team_record = team_data['Win-Loss Percentage'].values[0]
       
         fig.add_trace(go.Scatter(x=team_data['Scoring Percentage'], y=team_data['Points Differential'],
                                  mode='markers', name=team, 
                                  marker=dict(size=team_data['Win-Loss Percentage'] * 100,
                                               color=team_color),
                                  hovertext=f"{team}' record: {team_record}"))
         
       fig.update_layout(title=f"Scoring Percentage & Points Differential for Teams in {selected_year}",
                       xaxis_title="Scoring Percentage",
                       yaxis_title="Points Differential",
                       legend_title="Team",
                       height=600)
       
       fig.update_yaxes(range=[-299, 399])
       fig.update_xaxes(range=[13, 59])
    
    fig.show()

    display(year_dropdown_5)
    display(metrics_dropdown_5)

def on_dropdown_change(change):
    update_plot_5(year_dropdown_5.value, metrics_dropdown_5.value)

year_dropdown_5.observe(on_dropdown_change, names='value')
metrics_dropdown_5.observe(on_dropdown_change, names='value')

display(year_dropdown_5)
display(metrics_dropdown_5)

Dropdown(description='Year:', index=1, options=(2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 20…

Dropdown(description='Metric:', options=('Expected Points', 'Scoring Percentage'), value='Expected Points')

In [408]:
team_dropdown_17 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

def update_plot_17(selected_team):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]

    team_points_color = points_color_map.get(f"{selected_team}_Points")
    team_points_allowed_color = points_color_map.get(f"{selected_team}_Points Allowed")

    fig = go.Figure()

    fig.add_trace(go.Box(y=filtered_data['Points'], x=['Points'] * len(filtered_data), 
                         name='Points Scored', marker_color=team_points_color, boxmean='sd'))

    fig.add_trace(go.Box(y=filtered_data['Points Allowed'], x=['Points Allowed'] * len(filtered_data), 
                         name='Points Allowed', marker_color=team_points_allowed_color, boxmean='sd'))

    fig.update_layout(title=f'Scoring Performance Spread for the {selected_team}', 
                      yaxis=dict(title='Points'),
                      xaxis=dict(title='', showticklabels=False),
                      boxmode='group', legend_title_text='')

    fig.show()
    display(team_dropdown_17)

team_dropdown_17.observe(lambda change: update_plot_17(change['new']), names='value')

display(team_dropdown_17)

Dropdown(description='Team:', index=1, options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'Ne…

In [409]:
NFL_stats['super_bowl_category'] = NFL_stats['SuperBowl Win'].apply(lambda x: 'Super Bowl Winners' if x > 0 else 'Non-Winners')

fig = px.box(NFL_stats, x='super_bowl_category', y='Points Differential', points='all', 
             color='super_bowl_category', 
             color_discrete_map={'Super Bowl Winners': 'blue', 'Non-Winners': 'red'},
             title='Points Differential Distribution: Super Bowl Winners vs Non-Winners')

fig.update_layout(
    xaxis_title='', 
    yaxis_title='Points Differential',
    xaxis={'categoryorder': 'array', 'categoryarray': ['Super Bowl Winners', 'Non-Winners']},
    legend_title=''
)

fig.show()

In [410]:
super_bowl_winners = NFL_stats[NFL_stats['SuperBowl Win'] > 0]

sorted_winners = super_bowl_winners.sort_values(by=['SuperBowl Win', 'Points Differential'], ascending=[False, False])

fig = px.bar(sorted_winners, x='Team', y='Points Differential', color='Points Differential', 
             title='Points Differential of Super Bowl Winners', 
             color_continuous_scale='Bluered_r')

fig.update_layout(yaxis_title='Sum of Points Differential',
                  xaxis={'categoryorder':'total ascending'}, xaxis_title='',
                  height=600)

fig.show()

In [411]:
NFL_stats['Super Bowl Winner'] = NFL_stats['SuperBowl Win'].map({0: 'No', 1: 'Yes'})
NFL_stats_sorted = NFL_stats.sort_values(by='Super Bowl Winner')

fig = px.scatter(NFL_stats_sorted, x='Points', y='Points Allowed', color='Super Bowl Winner', 
                 title="Points Scored vs Points Allowed Trend",
                 labels={'Points': 'Points Scored'},
                 color_discrete_sequence=px.colors.sequential.Bluered_r,
                 symbol='Super Bowl Winner',
                 symbol_sequence=["circle", "square"],
                 trendline='ols')

fig.update_layout(
    height=600,
    xaxis_title='Points Scored',
    yaxis_title='Points Allowed'
)

fig.show()

In [412]:
points_correlation_df = NFL_stats[['Points', 'Points Allowed', 'Margin of Victory', 'Expected Points Total', 
                                   'Scoring Percentage', 'SuperBowl Win']].rename(
    columns={'Points': 'Points Scored'})


points_correlation_matrix = points_correlation_df.corr()

fig = px.imshow(points_correlation_matrix, 
                text_auto=True, 
                color_continuous_scale='bluered_r', 
                zmin=-1, zmax=1, 
                labels={'color': 'Correlation'},
                title='Correlation Between Scoring Efficiency and Championship Success')

fig.update_layout(height=650)

fig.show()

In [413]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Points']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Points']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in points scored between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in points scored between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 10562.0, P-Value = 2.0873632694880724e-05, There is a significant difference in points scored between teams that won and teams that did not win the Super Bowl.


In [414]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Points Allowed']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Points Allowed']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in points allowed between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in points allowed between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 3453.0, P-Value = 0.00011223844308018927, There is a significant difference in points allowed between teams that won and teams that did not win the Super Bowl.


In [415]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Margin of Victory']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Margin of Victory']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in MOV between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in MOV between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 11584.0, P-Value = 5.878718048852112e-08, There is a significant difference in MOV between teams that won and teams that did not win the Super Bowl.


In [416]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Expected Points Total']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Expected Points Total']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in expected points between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in expected points between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 9268.0, P-Value = 0.005478694939639145, There is a significant difference in expected points between teams that won and teams that did not win the Super Bowl.


In [417]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Scoring Percentage']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Scoring Percentage']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in scoring percentage between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in scoring percentage between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 10076.5, P-Value = 0.0002148551370718693, There is a significant difference in scoring percentage between teams that won and teams that did not win the Super Bowl.


There is not a strong linear relationship between scoring efficiency variables and Super Bowl wins. That is, having a better scoring efficiency is not directly associated in a linear fashion with winning the Super Bowl. Still, there is a difference in the distribution of it. Super Bowl winning teams tend to have better scoring efficiency, but the relationship is not strong enough to be reflected in a linear correlation.

#### 2.2.2. <a id='toc2_2_2_'></a>[Passing vs. Rushing](#toc0_)

In [418]:
offensive_attempts_color_map = {
    'New England Patriots_Passing': '#002244', 'New England Patriots_Rushing': '#C60C30',
    'Miami Dolphins_Passing': '#008E97', 'Miami Dolphins_Rushing': '#FC4C02',
    'Buffalo Bills_Passing': '#00338D', 'Buffalo Bills_Rushing': '#C60C30',
    'New York Jets_Passing': '#1F9870', 'New York Jets_Rushing': '#000000',
    'Baltimore Ravens_Passing': '#3925BB', 'Baltimore Ravens_Rushing': '#000000',
    'Cincinnati Bengals_Passing': '#FB4F14', 'Cincinnati Bengals_Rushing': '#000000',
    'Cleveland Browns_Passing': '#311D00', 'Cleveland Browns_Rushing': '#FF3C00',
    'Pittsburgh Steelers_Passing': '#F5AB00', 'Pittsburgh Steelers_Rushing': '#000000',
    'Houston Texans_Passing': '#03202F', 'Houston Texans_Rushing': '#D5203E',
    'Indianapolis Colts_Passing': '#002C5F', 'Indianapolis Colts_Rushing': '#889296',
    'Jacksonville Jaguars_Passing': '#00ADCC', 'Jacksonville Jaguars_Rushing': '#000000',
    'Tennessee Titans_Passing': '#0C2340', 'Tennessee Titans_Rushing': '#4B92DB',
    'Denver Broncos_Passing': '#002244', 'Denver Broncos_Rushing': '#FB4F14',
    'Kansas City Chiefs_Passing': '#E31837', 'Kansas City Chiefs_Rushing': '#F5AB00',
    'Las Vegas Raiders_Passing': '#000000', 'Las Vegas Raiders_Rushing': '#9EA5A9',
    'Los Angeles Chargers_Passing': '#0080C6', 'Los Angeles Chargers_Rushing': '#FFC20E',
    'Chicago Bears_Passing': '#0B162A', 'Chicago Bears_Rushing': '#C83803',
    'Detroit Lions_Passing': '#0076B6', 'Detroit Lions_Rushing': '#869098',
    'Green Bay Packers_Passing': '#203731', 'Green Bay Packers_Rushing': '#F5AB00',
    'Minnesota Vikings_Passing': '#4F2683', 'Minnesota Vikings_Rushing': '#F5AB00',
    'Dallas Cowboys_Passing': '#003594', 'Dallas Cowboys_Rushing': '#919EA1',
    'New York Giants_Passing': '#0B2265', 'New York Giants_Rushing': '#D5203E',
    'Philadelphia Eagles_Passing': '#00373D', 'Philadelphia Eagles_Rushing': '#919EA1',
    'Washington Commanders_Passing': '#5A1414', 'Washington Commanders_Rushing': '#F5AB00',
    'Atlanta Falcons_Passing': '#D5203E', 'Atlanta Falcons_Rushing': '#000000',
    'Carolina Panthers_Passing': '#0085CA', 'Carolina Panthers_Rushing': '#000000',
    'New Orleans Saints_Passing': '#C6A86C', 'New Orleans Saints_Rushing': '#000000',
    'Tampa Bay Buccaneers_Passing': '#AF0808', 'Tampa Bay Buccaneers_Rushing': '#FF9233',
    'Arizona Cardinals_Passing': '#C62F52', 'Arizona Cardinals_Rushing': '#000000',
    'Los Angeles Rams_Passing': '#003594', 'Los Angeles Rams_Rushing': '#FFA300',
    'San Francisco 49ers_Passing': '#8F0000', 'San Francisco 49ers_Rushing': '#C4B082',
    'Seattle Seahawks_Passing': '#002244', 'Seattle Seahawks_Rushing': '#69BE28'
}

In [457]:
metrics_dropdown_23 = widgets.Dropdown(
    options=["Sum of Pass Yards & Success", "Total Pass Yards Comparison"],
    description="Metric:"
)

NFL_stats_sorted = NFL_stats.sort_values(by='Win-Loss Percentage', ascending=False)

total_pass_yards = NFL_stats.groupby('Team', as_index=False)['Passing Yards'].sum()

total_pass_yards = total_pass_yards.sort_values(by='Passing Yards', ascending=False)

def update_plot_23(selected_metric):
    clear_output(wait=True)

    if selected_metric == "Sum of Pass Yards & Success":
        fig = px.bar(NFL_stats_sorted, x='Team', y='Passing Yards', color='Win-Loss Percentage', barmode='group',
                     title="Passing Yards vs Win-Loss Percentage",
                     color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='',  
            coloraxis_colorbar=dict(title="Win-Loss %"),
            height=700
        )

    elif selected_metric == "Total Pass Yards Comparison":
        fig = px.bar(total_pass_yards, 
             x='Team', 
             y='Passing Yards', 
             title="Total Passing Yards Comparison",
             color='Passing Yards',
             color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='', 
            yaxis_title='Passing Yards',
            height=700,
            coloraxis_showscale=False
        )

    fig.show()
    display(metrics_dropdown_23)

metrics_dropdown_23.observe(lambda change: update_plot_23(change['new']), names='value')
display(metrics_dropdown_23)

Dropdown(description='Metric:', index=1, options=('Sum of Pass Yards & Success', 'Total Pass Yards Comparison'…

In [459]:
metrics_dropdown_24 = widgets.Dropdown(
    options=["Sum of Pass Attempts & Success", "Total Pass Attempts Comparison"],
    description="Metric:"
)

NFL_stats_sorted = NFL_stats.sort_values(by='Win-Loss Percentage', ascending=False)

total_pass_attempts = NFL_stats.groupby('Team', as_index=False)['Pass Attempts'].sum()

total_pass_attempts = total_pass_attempts.sort_values(by='Pass Attempts', ascending=False)

def update_plot_24(selected_metric):
    clear_output(wait=True)

    if selected_metric == "Sum of Pass Attempts & Success":
        fig = px.bar(NFL_stats_sorted, x='Team', y='Pass Attempts', color='Win-Loss Percentage', barmode='group',
                     title="Pass Attempts vs Win-Loss Percentage",
                     color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='',  
            coloraxis_colorbar=dict(title="Win-Loss %"),
            height=700
        )

    elif selected_metric == "Total Pass Attempts Comparison":
        fig = px.bar(total_pass_attempts, 
             x='Team', 
             y='Pass Attempts', 
             title="Total Pass Attempts Comparison",
             color='Pass Attempts',
             color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='', 
            yaxis_title='Pass Attempts',
            height=700,
            coloraxis_showscale=False
        )

    fig.show()
    display(metrics_dropdown_24)

metrics_dropdown_24.observe(lambda change: update_plot_24(change['new']), names='value')
display(metrics_dropdown_24)

Dropdown(description='Metric:', options=('Sum of Pass Attempts & Success', 'Total Pass Attempts Comparison'), …

In [461]:
metrics_dropdown_25 = widgets.Dropdown(
    options=["Sum of Rush Yards & Success", "Total Rush Yards Comparison"],
    description="Metric:"
)

NFL_stats_sorted = NFL_stats.sort_values(by='Win-Loss Percentage', ascending=False)

total_rush_yards = NFL_stats.groupby('Team', as_index=False)['Rushing Yards'].sum()

total_rush_yards = total_rush_yards.sort_values(by='Rushing Yards', ascending=False)

def update_plot_25(selected_metric):
    clear_output(wait=True)

    if selected_metric == "Sum of Rush Yards & Success":
        fig = px.bar(NFL_stats_sorted, x='Team', y='Rushing Yards', color='Win-Loss Percentage', barmode='group',
                     title="Rushing Yards vs Win-Loss Percentage",
                     color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='',  
            coloraxis_colorbar=dict(title="Win-Loss %"),
            height=700
        )

    elif selected_metric == "Total Rush Yards Comparison":
        fig = px.bar(total_rush_yards, 
             x='Team', 
             y='Rushing Yards', 
             title="Total Rushing Yards Comparison",
             color='Rushing Yards',
             color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='', 
            yaxis_title='Rushing Yards',
            height=700,
            coloraxis_showscale=False
        )

    fig.show()
    display(metrics_dropdown_25)

metrics_dropdown_25.observe(lambda change: update_plot_25(change['new']), names='value')
display(metrics_dropdown_25)

Dropdown(description='Metric:', options=('Sum of Rush Yards & Success', 'Total Rush Yards Comparison'), value=…

In [462]:
metrics_dropdown_26 = widgets.Dropdown(
    options=["Sum of Rush Attempts & Success", "Total Rush Attempts Comparison"],
    description="Metric:"
)

NFL_stats_sorted = NFL_stats.sort_values(by='Win-Loss Percentage', ascending=False)

total_rush_attempts = NFL_stats.groupby('Team', as_index=False)['Rush Attempts'].sum()

total_rush_attempts = total_rush_attempts.sort_values(by='Rush Attempts', ascending=False)

def update_plot_26(selected_metric):
    clear_output(wait=True)

    if selected_metric == "Sum of Rush Attempts & Success":
        fig = px.bar(NFL_stats_sorted, x='Team', y='Rush Attempts', color='Win-Loss Percentage', barmode='group',
                     title="Rushing Attempts vs Win-Loss Percentage",
                     color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='',  
            coloraxis_colorbar=dict(title="Win-Loss %"),
            height=700
        )

    elif selected_metric == "Total Rush Attempts Comparison":
        fig = px.bar(total_rush_attempts, 
             x='Team', 
             y='Rush Attempts', 
             title="Total Rushing Attempts Comparison",
             color='Rush Attempts',
             color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='', 
            yaxis_title='Rush Attempts',
            height=700,
            coloraxis_showscale=False
        )

    fig.show()
    display(metrics_dropdown_26)

metrics_dropdown_26.observe(lambda change: update_plot_26(change['new']), names='value')
display(metrics_dropdown_26)

Dropdown(description='Metric:', index=1, options=('Sum of Rush Attempts & Success', 'Total Rush Attempts Compa…

In [419]:
team_dropdown_6 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metrics_dropdown_6 = widgets.Dropdown(
    options=["Attempts & Success", "Attempts Comparison"],
    description="Metric:"
)

def update_plot_6(selected_team, selected_metric):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]

    fig = go.Figure()

    if selected_metric == "Attempts & Success":

        hover_text = [
            f"Year: {year}<br>Win-Loss %: {win_loss_pct}" 
            for year, win_loss_pct in zip(filtered_data['Year'], filtered_data['Win-Loss Percentage'])
        ]

        fig.add_trace(go.Scatter(x=filtered_data['Pass Attempts'], y=filtered_data['Rush Attempts'],
                                  mode='markers', hovertext=hover_text,
                                  marker=dict(color=filtered_data['Win-Loss Percentage'],
                                              colorscale='Bluered_r',
                                              colorbar=dict(title="Win/Loss %"),
                                              size=15)))
        
        fig.update_layout(title=f"{selected_team}' Passing vs Rushing Attempts by Success",
                          xaxis_title="Pass Attempts",
                          yaxis_title="Rush Attempts")

    elif selected_metric == "Attempts Comparison":

        pass_att_color = offensive_attempts_color_map.get(f"{selected_team}_Passing")
        rush_att_color = offensive_attempts_color_map.get(f"{selected_team}_Rushing")

        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Pass Attempts'], 
                                 mode='lines', name='Pass Attempts',
                                 line=dict(color=pass_att_color)))
        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Rush Attempts'], 
                                 mode='lines', name='Rush Attempts',
                                 line=dict(color=rush_att_color)))

        fig.update_layout(title=f"Passing vs Rushing Attempts for the {selected_team}",
                          xaxis_title="Year",
                          yaxis_title="Attempts",
                          legend_title="")
    
    fig.show()

    display(team_dropdown_6)
    display(metrics_dropdown_6)

def on_dropdown_change(change):
    update_plot_6(team_dropdown_6.value, metrics_dropdown_6.value)

team_dropdown_6.observe(on_dropdown_change, names='value')
metrics_dropdown_6.observe(on_dropdown_change, names='value')

display(team_dropdown_6)
display(metrics_dropdown_6)

Dropdown(description='Team:', options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'New York Je…

Dropdown(description='Metric:', options=('Attempts & Success', 'Attempts Comparison'), value='Attempts & Succe…

In [420]:
team_dropdown_7 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metrics_dropdown_7 = widgets.Dropdown(
    options=["Yards & Success", "Yards Comparison"],
    description="Metric:"
)

def update_plot_7(selected_team, selected_metric):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]

    fig = go.Figure()

    if selected_metric == "Yards & Success":

        hover_text = [
            f"Year: {year}<br>Win-Loss %: {win_loss_pct}" 
            for year, win_loss_pct in zip(filtered_data['Year'], filtered_data['Win-Loss Percentage'])
        ]

        fig.add_trace(go.Scatter(x=filtered_data['Passing Yards'], y=filtered_data['Rushing Yards'],
                                  mode='markers', hovertext=hover_text,
                                  marker=dict(color=filtered_data['Win-Loss Percentage'],
                                              colorscale='Bluered_r',
                                              colorbar=dict(title="Win/Loss %"),
                                              size=15)))
        
        fig.update_layout(title=f"{selected_team}' Passing vs Rushing Yards by Success",
                          xaxis_title="Passing Yards",
                          yaxis_title="Rushing Yards")

    elif selected_metric == "Yards Comparison":

        pass_att_color = offensive_attempts_color_map.get(f"{selected_team}_Passing")
        rush_att_color = offensive_attempts_color_map.get(f"{selected_team}_Rushing")

        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Passing Yards'], 
                                 mode='lines', name='Passing Yards',
                                 line=dict(color=pass_att_color)))
        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Rushing Yards'], 
                                 mode='lines', name='Rushing Yards',
                                 line=dict(color=rush_att_color)))

        fig.update_layout(title=f"Passing vs Rushing Yards for the {selected_team}",
                          xaxis_title="Year",
                          yaxis_title="Yards",
                          legend_title="")
    
    fig.show()

    display(team_dropdown_7)
    display(metrics_dropdown_7)

def on_dropdown_change(change):
    update_plot_7(team_dropdown_7.value, metrics_dropdown_7.value)

team_dropdown_7.observe(on_dropdown_change, names='value')
metrics_dropdown_7.observe(on_dropdown_change, names='value')

display(team_dropdown_7)
display(metrics_dropdown_7)

Dropdown(description='Team:', options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'New York Je…

Dropdown(description='Metric:', options=('Yards & Success', 'Yards Comparison'), value='Yards & Success')

In [468]:
team_dropdown_27 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metric_group_dropdown_27 = widgets.Dropdown(
    options=["Yards", "Attempts"],  
    description="Metric:"
)

metric_mapping = {
    "Yards": {
        "Passing Yards": 'Passing Yards',
        "Rushing Yards": 'Rushing Yards'
    },
    "Attempts": {
        "Pass Attempts": 'Pass Attempts',
        "Rush Attempts": 'Rush Attempts'
    }
}

def update_plot_27(selected_team, selected_metric_group):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]
    
    metrics = metric_mapping[selected_metric_group]
    y_axis_label = 'Yards' if selected_metric_group == "Yards" else 'Attempts'
    
    fig = px.box()
    
    for metric_name, y_column in metrics.items():
        if y_column in filtered_data.columns:
            if "Passing" in metric_name or "Pass" in metric_name:
                team_color = home_road_color_map.get(f"{selected_team}_Home")
            else:
                team_color = home_road_color_map.get(f"{selected_team}_Road")

            trace = px.box(
                filtered_data, 
                x='Team', 
                y=y_column, 
                labels={y_column: y_axis_label},
                color_discrete_sequence=[team_color]
            ).data[0]
            
            trace.name = metric_name
            trace.showlegend = True
            fig.add_trace(trace)

    fig.update_layout(
        title=f"{selected_metric_group.capitalize()} Spread for the {selected_team}",
        yaxis_title=y_axis_label,
        xaxis_title='',
        showlegend=True, 
        legend_title_text='',
        xaxis=dict(showticklabels=False)
    )
    
    fig.show()
    
    display(team_dropdown_27)
    display(metric_group_dropdown_27)

def handle_change(change):
    update_plot_27(team_dropdown_27.value, metric_group_dropdown_27.value)

team_dropdown_27.observe(handle_change, names='value')
metric_group_dropdown_27.observe(handle_change, names='value')

display(team_dropdown_27)
display(metric_group_dropdown_27)

Dropdown(description='Team:', index=8, options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'Ne…

Dropdown(description='Metric:', options=('Yards', 'Attempts'), value='Yards')

In [421]:
team_dropdown_8 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metrics_dropdown_8 = widgets.Dropdown(
    options=["Touchdowns & Success", "Touchdowns Comparison"],
    description="Metric:"
)

def update_plot_8(selected_team, selected_metric):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]

    fig = go.Figure()

    if selected_metric == "Touchdowns & Success":

        hover_text = [
            f"Year: {year}<br>Win-Loss %: {win_loss_pct}" 
            for year, win_loss_pct in zip(filtered_data['Year'], filtered_data['Win-Loss Percentage'])
        ]

        fig.add_trace(go.Scatter(x=filtered_data['Passing TD'], y=filtered_data['Rushing TD'],
                                  mode='markers', hovertext=hover_text,
                                  marker=dict(color=filtered_data['Win-Loss Percentage'],
                                              colorscale='Bluered_r',
                                              colorbar=dict(title="Win/Loss %"),
                                              size=15)))
        
        fig.update_layout(title=f"{selected_team}' Passing vs Rushing Touchdowns by Success",
                          xaxis_title="Passing Touchdowns",
                          yaxis_title="Rushing Touchdowns")

    elif selected_metric == "Touchdowns Comparison":

        pass_att_color = offensive_attempts_color_map.get(f"{selected_team}_Passing")
        rush_att_color = offensive_attempts_color_map.get(f"{selected_team}_Rushing")

        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Passing TD'], 
                                 mode='lines', name='Passing Touchdowns',
                                 line=dict(color=pass_att_color)))
        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Rushing TD'], 
                                 mode='lines', name='Rushing Touchdowns',
                                 line=dict(color=rush_att_color)))

        fig.update_layout(title=f"Passing vs Rushing Touchdowns for the {selected_team}",
                          xaxis_title="Year",
                          yaxis_title="Touchdowns",
                          legend_title="")
    
    fig.show()

    display(team_dropdown_8)
    display(metrics_dropdown_8)

def on_dropdown_change(change):
    update_plot_8(team_dropdown_8.value, metrics_dropdown_8.value)

team_dropdown_8.observe(on_dropdown_change, names='value')
metrics_dropdown_8.observe(on_dropdown_change, names='value')

display(team_dropdown_8)
display(metrics_dropdown_8)

Dropdown(description='Team:', options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'New York Je…

Dropdown(description='Metric:', options=('Touchdowns & Success', 'Touchdowns Comparison'), value='Touchdowns &…

In [422]:
team_dropdown_9 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metrics_dropdown_9 = widgets.Dropdown(
    options=["First Downs & Success", "First Downs Comparison"],
    description="Metric:"
)

def update_plot_9(selected_team, selected_metric):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]

    fig = go.Figure()

    if selected_metric == "First Downs & Success":

        hover_text = [
            f"Year: {year}<br>Win-Loss %: {win_loss_pct}" 
            for year, win_loss_pct in zip(filtered_data['Year'], filtered_data['Win-Loss Percentage'])
        ]

        fig.add_trace(go.Scatter(x=filtered_data['Passing First Downs'], y=filtered_data['Rushing First Downs'],
                                  mode='markers', hovertext=hover_text,
                                  marker=dict(color=filtered_data['Win-Loss Percentage'],
                                              colorscale='Bluered_r',
                                              colorbar=dict(title="Win/Loss %"),
                                              size=15)))
        
        fig.update_layout(title=f"{selected_team}' Passing vs Rushing First Downs by Success",
                          xaxis_title="Passing First Downs",
                          yaxis_title="Rushing First Downs")

    elif selected_metric == "First Downs Comparison":

        pass_att_color = offensive_attempts_color_map.get(f"{selected_team}_Passing")
        rush_att_color = offensive_attempts_color_map.get(f"{selected_team}_Rushing")

        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Passing First Downs'], 
                                 mode='lines', name='Passing First Downs',
                                 line=dict(color=pass_att_color)))
        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Rushing First Downs'], 
                                 mode='lines', name='Rushing First Downs',
                                 line=dict(color=rush_att_color)))

        fig.update_layout(title=f"Passing vs Rushing First Downs for the {selected_team}",
                          xaxis_title="Year",
                          yaxis_title="First Downs",
                          legend_title="")
    
    fig.show()

    display(team_dropdown_9)
    display(metrics_dropdown_9)

def on_dropdown_change(change):
    update_plot_9(team_dropdown_9.value, metrics_dropdown_9.value)

team_dropdown_9.observe(on_dropdown_change, names='value')
metrics_dropdown_9.observe(on_dropdown_change, names='value')

display(team_dropdown_9)
display(metrics_dropdown_9)

Dropdown(description='Team:', options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'New York Je…

Dropdown(description='Metric:', options=('First Downs & Success', 'First Downs Comparison'), value='First Down…

In [423]:
team_dropdown_10 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metrics_dropdown_10 = widgets.Dropdown(
    options=["Yards per Play & Success", "Yards per Play Comparison"],
    description="Metric:"
)

def update_plot_10(selected_team, selected_metric):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]

    fig = go.Figure()

    if selected_metric == "Yards per Play & Success":

        hover_text = [
            f"Year: {year}<br>Win-Loss %: {win_loss_pct}" 
            for year, win_loss_pct in zip(filtered_data['Year'], filtered_data['Win-Loss Percentage'])
        ]

        team_color = primary_team_colors.get(selected_team)

        fig.add_trace(go.Scatter(x=filtered_data['Win-Loss Percentage'], y=filtered_data['Total Yards per Play'],
                                  mode='markers', hovertext=hover_text,
                                  marker=dict(color=team_color,
                                              size=15)))
        
        fig.update_layout(title=f"{selected_team}' Yards per Play vs Win-Loss Percentage",
                          xaxis_title="Win-Loss Percentage",
                          yaxis_title="Yards per Play")
        
    elif selected_metric == "Yards per Play Comparison":

        pass_att_color = offensive_attempts_color_map.get(f"{selected_team}_Passing")
        rush_att_color = offensive_attempts_color_map.get(f"{selected_team}_Rushing")
        team_color = primary_team_colors.get(selected_team)

        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Passing Yards per Attempt'], 
                                 mode='lines', name='Passing Yards',
                                 line=dict(color=pass_att_color)))
        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Rushing Yards per Attempt'], 
                                 mode='lines', name='Rushing Yards',
                                 line=dict(color=rush_att_color)))
        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Total Yards per Play'], 
                                 mode='markers', name='Total Yards',
                                 marker=dict(color=team_color)))

        fig.update_layout(title=f"Passing vs Rushing Yards per Attempt for the {selected_team}",
                          xaxis_title="Year",
                          yaxis_title="Yards per Attempt",
                          legend_title="")
    
    fig.show()

    display(team_dropdown_10)
    display(metrics_dropdown_10)

def on_dropdown_change(change):
    update_plot_10(team_dropdown_10.value, metrics_dropdown_10.value)

team_dropdown_10.observe(on_dropdown_change, names='value')
metrics_dropdown_10.observe(on_dropdown_change, names='value')

display(team_dropdown_10)
display(metrics_dropdown_10)

Dropdown(description='Team:', options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'New York Je…

Dropdown(description='Metric:', options=('Yards per Play & Success', 'Yards per Play Comparison'), value='Yard…

In [473]:
super_bowl_winners = NFL_stats[NFL_stats['SuperBowl Win'] > 0]

sorted_winners = super_bowl_winners.sort_values(by=['SuperBowl Win', 'Passing Yards'], ascending=[False, False])

fig = px.bar(sorted_winners, x='Team', y='Passing Yards', color='Passing Yards', 
             title='Passing Yards of Super Bowl Winners', 
             color_continuous_scale='Bluered_r', hover_data={'Year': True})

fig.update_layout(yaxis_title='Passing Yards',
                  xaxis={'categoryorder':'total descending'}, xaxis_title='',
                  height=600)

fig.show()

In [472]:
super_bowl_winners = NFL_stats[NFL_stats['SuperBowl Win'] > 0]

sorted_winners = super_bowl_winners.sort_values(by=['SuperBowl Win', 'Rushing Yards'], ascending=[False, False])

fig = px.bar(sorted_winners, x='Team', y='Rushing Yards', color='Rushing Yards', 
             title='Rushing Yards of Super Bowl Winners', 
             color_continuous_scale='Bluered_r', hover_data={'Year': True})

fig.update_layout(yaxis_title='Rushing Yards',
                  xaxis={'categoryorder':'total descending'}, xaxis_title='',
                  height=600)

fig.show()

In [424]:
points_correlation_df = NFL_stats[['Pass Attempts', 'Passing Yards', 'Passing TD', 'Passing First Downs', 
                                   'Passing Yards per Attempt', 'Total Yards per Play', 'SuperBowl Win']]


points_correlation_matrix = points_correlation_df.corr()

fig = px.imshow(points_correlation_matrix, 
                text_auto=True, 
                color_continuous_scale='bluered_r', 
                zmin=-1, zmax=1, 
                labels={'color': 'Correlation'},
                title='Correlation Between Passing Success and Championship Success')

fig.update_layout(height=700)

fig.show()

In [425]:
points_correlation_df = NFL_stats[['Rush Attempts', 'Rushing Yards', 'Rushing TD', 'Rushing First Downs', 
                                   'Rushing Yards per Attempt', 'Total Yards per Play', 'SuperBowl Win']]


points_correlation_matrix = points_correlation_df.corr()

fig = px.imshow(points_correlation_matrix, 
                text_auto=True, 
                color_continuous_scale='bluered_r', 
                zmin=-1, zmax=1, 
                labels={'color': 'Correlation'},
                title='Correlation Between Rushing Success and Championship Success')

fig.update_layout(height=700)

fig.show()

In [426]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Pass Attempts']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Pass Attempts']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in Pass Attempts between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in Pass Attempts between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 7807.5, P-Value = 0.26720506985885184, There is no significant difference in Pass Attempts between teams that won and teams that did not win the Super Bowl.


In [427]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Rush Attempts']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Rush Attempts']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in Rush Attempts between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in Rush Attempts between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 7932.0, P-Value = 0.2106717747661616, There is no significant difference in Rush Attempts between teams that won and teams that did not win the Super Bowl.


In [428]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Passing Yards']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Passing Yards']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in Passing Yards between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in Passing Yards between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 9428.5, P-Value = 0.0030688757001542475, There is a significant difference in Passing Yards between teams that won and teams that did not win the Super Bowl.


In [429]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Rushing Yards']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Rushing Yards']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in Rushing Yards between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in Rushing Yards between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 7139.0, P-Value = 0.7293125047918461, There is no significant difference in Rushing Yards between teams that won and teams that did not win the Super Bowl.


In [430]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Passing TD']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Passing TD']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in passing touchdowns between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in passing touchdowns between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 10109.0, P-Value = 0.00018302090611180407, There is a significant difference in passing touchdowns between teams that won and teams that did not win the Super Bowl.


In [431]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Rushing TD']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Rushing TD']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in rushing touchdowns between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in rushing touchdowns between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 8417.5, P-Value = 0.0703692626309691, There is no significant difference in rushing touchdowns between teams that won and teams that did not win the Super Bowl.


In [432]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Passing First Downs']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Passing First Downs']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in Passing First Downs between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in Passing First Downs between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 9076.5, P-Value = 0.010499959103166995, There is a significant difference in Passing First Downs between teams that won and teams that did not win the Super Bowl.


In [433]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Rushing First Downs']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Rushing First Downs']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in Rushing First Downs between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in Rushing First Downs between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 7951.0, P-Value = 0.2028231612096707, There is no significant difference in Rushing First Downs between teams that won and teams that did not win the Super Bowl.


In [434]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Passing Yards per Attempt']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Passing Yards per Attempt']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in Passing Yards per Attempt between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in Passing Yards per Attempt between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 10339.0, P-Value = 6.23561143475118e-05, There is a significant difference in Passing Yards per Attempt between teams that won and teams that did not win the Super Bowl.


In [435]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Rushing Yards per Attempt']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Rushing Yards per Attempt']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in Rushing Yards per Attempt between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in Rushing Yards per Attempt between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 6147.0, P-Value = 0.4308177129614783, There is no significant difference in Rushing Yards per Attempt between teams that won and teams that did not win the Super Bowl.


There is not a strong linear relationship between rushing variables and Super Bowl wins, and neither a difference in the distribution of them.

On the other hand, while there isn't a strong linear relationship between passing variables and Super Bowl wins, there is a difference in the distribution of them (except for passing attempts). Super Bowl winning teams tend to have better passing offense.

Regarding Total Yards per Play, as we can see in the matrix, there is a stronger linear relationship with passing variables compared to rushing ones.

### 2.3. <a id='toc2_3_'></a>[Turnover Battle & Its Influence on Wins](#toc0_)

In [436]:
metrics_dropdown_11 = widgets.Dropdown(
    options=["Sum of INT & Success", "Total INT Comparison"],
    description="Metric:"
)

NFL_stats_sorted = NFL_stats.sort_values(by='Win-Loss Percentage', ascending=False)

total_interceptions = NFL_stats.groupby('Team', as_index=False)['Interceptions'].sum()

total_interceptions = total_interceptions.sort_values(by='Interceptions', ascending=True)

def update_plot_11(selected_metric):
    clear_output(wait=True)

    if selected_metric == "Sum of INT & Success":
        fig = px.bar(NFL_stats_sorted, x='Team', y='Interceptions', color='Win-Loss Percentage', barmode='group',
                     title="Interceptions vs Win-Loss Percentage",
                     color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='',  
            coloraxis_colorbar=dict(title="Win-Loss %"),
            height=700
        )

    elif selected_metric == "Total INT Comparison":
        fig = px.bar(total_interceptions, 
             x='Team', 
             y='Interceptions', 
             title="Total Interceptions Comparison",
             color='Interceptions',
             color_continuous_scale='Bluered')
        
        fig.update_layout(
            xaxis_title='', 
            yaxis_title='Interceptions',
            height=700,
            coloraxis_showscale=False
        )

    fig.show()
    display(metrics_dropdown_11)

metrics_dropdown_11.observe(lambda change: update_plot_11(change['new']), names='value')
display(metrics_dropdown_11)

Dropdown(description='Metric:', index=1, options=('Sum of INT & Success', 'Total INT Comparison'), value='Tota…

In [437]:
metrics_dropdown_12 = widgets.Dropdown(
    options=["Sum of Fumbles & Success", "Total Fumbles Comparison"],
    description="Metric:"
)

NFL_stats_sorted = NFL_stats.sort_values(by='Win-Loss Percentage', ascending=False)

total_fumbles = NFL_stats.groupby('Team', as_index=False)['Fumbles Lost'].sum()

total_fumbles = total_fumbles.sort_values(by='Fumbles Lost', ascending=True)

def update_plot_12(selected_metric):
    clear_output(wait=True)

    if selected_metric == "Sum of Fumbles & Success":
        fig = px.bar(NFL_stats_sorted, x='Team', y='Fumbles Lost', color='Win-Loss Percentage', barmode='group',
                     title="Fumbles Lost vs Win-Loss Percentage",
                     color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='',  
            coloraxis_colorbar=dict(title="Win-Loss %"),
            height=700
        )

    elif selected_metric == "Total Fumbles Comparison":
        fig = px.bar(total_fumbles, 
             x='Team', 
             y='Fumbles Lost', 
             title="Total Fumbles Comparison",
             color='Fumbles Lost',
             color_continuous_scale='Bluered')
        
        fig.update_layout(
            xaxis_title='', 
            yaxis_title='Fumbles Lost',
            height=700,
            coloraxis_showscale=False
        )

    fig.show()
    display(metrics_dropdown_12)

metrics_dropdown_12.observe(lambda change: update_plot_12(change['new']), names='value')
display(metrics_dropdown_12)

Dropdown(description='Metric:', index=1, options=('Sum of Fumbles & Success', 'Total Fumbles Comparison'), val…

In [438]:
metrics_dropdown_13 = widgets.Dropdown(
    options=["Sum of Turnovers & Success", "Total Turnovers Comparison"],
    description="Metric:"
)

NFL_stats_sorted = NFL_stats.sort_values(by='Win-Loss Percentage', ascending=False)

total_turnovers = NFL_stats.groupby('Team', as_index=False)['Turnovers'].sum()

total_turnovers = total_turnovers.sort_values(by='Turnovers', ascending=True)

def update_plot_13(selected_metric):
    clear_output(wait=True)

    if selected_metric == "Sum of Turnovers & Success":
        fig = px.bar(NFL_stats_sorted, x='Team', y='Turnovers', color='Win-Loss Percentage', barmode='group',
                     title="Total Turnovers vs Win-Loss Percentage",
                     color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='',  
            coloraxis_colorbar=dict(title="Win-Loss %"),
            height=700
        )

    elif selected_metric == "Total Turnovers Comparison":
        fig = px.bar(total_turnovers, 
             x='Team', 
             y='Turnovers', 
             title="Total Turnovers Comparison",
             color='Turnovers',
             color_continuous_scale='Bluered')
        
        fig.update_layout(
            xaxis_title='', 
            yaxis_title='Turnovers',
            height=700,
            coloraxis_showscale=False
        )

    fig.show()
    display(metrics_dropdown_13)

metrics_dropdown_13.observe(lambda change: update_plot_13(change['new']), names='value')
display(metrics_dropdown_13)

Dropdown(description='Metric:', index=1, options=('Sum of Turnovers & Success', 'Total Turnovers Comparison'),…

In [439]:
team_dropdown_14 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metrics_dropdown_14 = widgets.Dropdown(
    options=["INT/FUM & Success", "INT/FUM Through Seasons"],
    description="Metric:"
)

def update_plot_14(selected_team, selected_metric):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]

    fig = go.Figure()

    if selected_metric == "INT/FUM & Success":

        hover_text = [
            f"Year: {year}<br>Win-Loss %: {win_loss_pct}" 
            for year, win_loss_pct in zip(filtered_data['Year'], filtered_data['Win-Loss Percentage'])
        ]

        fig.add_trace(go.Scatter(x=filtered_data['Interceptions'], y=filtered_data['Fumbles Lost'],
                                  mode='markers', hovertext=hover_text,
                                  marker=dict(color=filtered_data['Win-Loss Percentage'],
                                              colorscale='Bluered_r',
                                              colorbar=dict(title="Win/Loss %"),
                                              size=15)))
        
        fig.update_layout(title=f"{selected_team}' Interceptions & Fumbles Lost vs Success",
                          xaxis_title="Interceptions",
                          yaxis_title="Fumbles")
        
    elif selected_metric == "INT/FUM Through Seasons":

        pass_att_color = offensive_attempts_color_map.get(f"{selected_team}_Passing")
        rush_att_color = offensive_attempts_color_map.get(f"{selected_team}_Rushing")
        team_color = primary_team_colors.get(selected_team)

        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Interceptions'], 
                                 mode='lines', name='Interceptions',
                                 line=dict(color=pass_att_color)))
        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Fumbles Lost'], 
                                 mode='lines', name='Fumbles Lost',
                                 line=dict(color=rush_att_color)))
        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Turnovers'], 
                                 mode='markers', name='Total Turnovers',
                                 marker=dict(color=team_color)))

        fig.update_layout(title=f"Turnovers Through Seasons for the {selected_team}",
                          xaxis_title="Year",
                          yaxis_title="Turnovers",
                          legend_title="")
    
    fig.show()

    display(team_dropdown_14)
    display(metrics_dropdown_14)

def on_dropdown_change(change):
    update_plot_14(team_dropdown_14.value, metrics_dropdown_14.value)

team_dropdown_14.observe(on_dropdown_change, names='value')
metrics_dropdown_14.observe(on_dropdown_change, names='value')

display(team_dropdown_14)
display(metrics_dropdown_14)

Dropdown(description='Team:', options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'New York Je…

Dropdown(description='Metric:', index=1, options=('INT/FUM & Success', 'INT/FUM Through Seasons'), value='INT/…

In [475]:
team_dropdown_28 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metrics_to_compare = {
    "Interceptions": 'Interceptions',
    "Fumbles Lost": 'Fumbles Lost'
}

def update_plot_28(selected_team):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]
    
    fig = px.box()

    for metric_name, y_column in metrics_to_compare.items():
        if y_column in filtered_data.columns:
            if metric_name == "Interceptions":
                team_color = home_road_color_map.get(f"{selected_team}_Home")
            else:
                team_color = home_road_color_map.get(f"{selected_team}_Road")

            trace = px.box(
                filtered_data, 
                x='Team', 
                y=y_column, 
                labels={y_column: 'Turnovers'},
                color_discrete_sequence=[team_color]
            ).data[0]
            
            trace.name = metric_name
            trace.showlegend = True
            fig.add_trace(trace)

    fig.update_layout(
        title=f"Spread of Interceptions & Fumbles Lost for the {selected_team}",
        yaxis_title='Turnovers',
        xaxis_title='',
        showlegend=True,
        legend_title_text='',
        xaxis=dict(showticklabels=False),
        yaxis=dict(range=[1,33]),
        height=600
    )
    
    fig.show()
    display(team_dropdown_28)

def handle_change(change):
    update_plot_28(team_dropdown_28.value)

team_dropdown_28.observe(handle_change, names='value')

display(team_dropdown_28)

Dropdown(description='Team:', index=23, options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'N…

In [440]:
year_dropdown_15 = widgets.Dropdown(
    options=NFL_stats['Year'].unique(), 
    description="Year:"
)

def update_plot_15(selected_year):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Year'] == selected_year]

    fig = go.Figure()


    for team in filtered_data['Team'].unique():
        team_data = filtered_data[filtered_data['Team'] == team]
        team_color = real_primary_team_colors.get(team)
        team_record = team_data['Win-Loss Percentage'].values[0]
        
        fig.add_trace(go.Scatter(x=team_data['Turnover Percentage'], y=team_data['Scoring Percentage'],
                                  mode='markers', name=team,
                                  marker=dict(size=team_data['Win-Loss Percentage'] * 100,
                                              color=team_color),
                                  hovertext=f"{team}' record: {team_record}"))

    fig.update_layout(title=f"Turnover vs Scoring Percentage for Teams in {selected_year}",
                       xaxis_title="Turnover Percentage",
                       yaxis_title="Scoring Percentage",
                       legend_title="Team",
                       height=600)
    
    fig.show()

    display(year_dropdown_15)

def on_dropdown_change(change):
    update_plot_15(year_dropdown_15.value)

year_dropdown_15.observe(on_dropdown_change, names='value')

display(year_dropdown_15)

Dropdown(description='Year:', index=1, options=(2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 20…

In [476]:
super_bowl_winners = NFL_stats[NFL_stats['SuperBowl Win'] > 0]

sorted_winners = super_bowl_winners.sort_values(by=['SuperBowl Win', 'Interceptions'], ascending=[False, False])

fig = px.bar(sorted_winners, x='Team', y='Interceptions', color='Interceptions', 
             title='Interceptions of Super Bowl Winners', 
             color_continuous_scale='Bluered', hover_data={'Year': True})

fig.update_layout(yaxis_title='Interceptions',
                  xaxis={'categoryorder':'total ascending'}, xaxis_title='',
                  height=600)

fig.show()

In [477]:
super_bowl_winners = NFL_stats[NFL_stats['SuperBowl Win'] > 0]

sorted_winners = super_bowl_winners.sort_values(by=['SuperBowl Win', 'Fumbles Lost'], ascending=[False, False])

fig = px.bar(sorted_winners, x='Team', y='Fumbles Lost', color='Fumbles Lost', 
             title='Fumbles Lost of Super Bowl Winners', 
             color_continuous_scale='Bluered', hover_data={'Year': True})

fig.update_layout(yaxis_title='Fumbles Lost',
                  xaxis={'categoryorder':'total ascending'}, xaxis_title='',
                  height=600)

fig.show()

In [478]:
super_bowl_winners = NFL_stats[NFL_stats['SuperBowl Win'] > 0]

sorted_winners = super_bowl_winners.sort_values(by=['SuperBowl Win', 'Turnovers'], ascending=[False, False])

fig = px.bar(sorted_winners, x='Team', y='Turnovers', color='Turnovers', 
             title='Total Turnovers of Super Bowl Winners', 
             color_continuous_scale='Bluered', hover_data={'Year': True})

fig.update_layout(yaxis_title='Turnovers',
                  xaxis={'categoryorder':'total ascending'}, xaxis_title='',
                  height=600)

fig.show()

In [441]:
points_correlation_df = NFL_stats[['Interceptions', 'Fumbles Lost', 'Turnovers',
                                   'Turnover Percentage', 'SuperBowl Win']]


points_correlation_matrix = points_correlation_df.corr()

fig = px.imshow(points_correlation_matrix, 
                text_auto=True, 
                color_continuous_scale='bluered_r', 
                zmin=-1, zmax=1, 
                labels={'color': 'Correlation'},
                title='Correlation Between Turnovers and Championship Success')

fig.update_layout(height=600)

fig.show()

In [442]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Interceptions']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Interceptions']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in Interceptions between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in Interceptions between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 5186.5, P-Value = 0.059227666279073665, There is no significant difference in Interceptions between teams that won and teams that did not win the Super Bowl.


In [443]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Fumbles Lost']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Fumbles Lost']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in fumbles lost between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in fumbles lost between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 6162.5, P-Value = 0.4406000324221433, There is no significant difference in fumbles lost between teams that won and teams that did not win the Super Bowl.


In [444]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Turnovers']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Turnovers']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in turnovers between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in turnovers between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 5206.0, P-Value = 0.06253221161775047, There is no significant difference in turnovers between teams that won and teams that did not win the Super Bowl.


In [445]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Turnover Percentage']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Turnover Percentage']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in turnover percentage between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in turnover percentage between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 4815.5, P-Value = 0.02108200421222936, There is a significant difference in turnover percentage between teams that won and teams that did not win the Super Bowl.


There is not a strong linear relationship between turnovers (both interceptions and fumbles) and Super Bowl wins, and neither a difference in the distribution of them.

However, there is a difference in the distribution of Turnover Percentage between Super Bowl winners and non-winners. The percentage of drives that result in a turnover tend to be lower for championship teams.

### 2.4. <a id='toc2_4_'></a>[Penalties and Discipline Impact on Success](#toc0_)

In [446]:
metrics_dropdown_18 = widgets.Dropdown(
    options=["Sum of Penalties & Success", "Total Penalties Comparison"],
    description="Metric:"
)

NFL_stats_sorted = NFL_stats.sort_values(by='Win-Loss Percentage', ascending=False)

total_penalties = NFL_stats.groupby('Team', as_index=False)['Penalties'].sum()

total_penalties = total_penalties.sort_values(by='Penalties', ascending=True)

def update_plot_18(selected_metric):
    clear_output(wait=True)

    if selected_metric == "Sum of Penalties & Success":
        fig = px.bar(NFL_stats_sorted, x='Team', y='Penalties', color='Win-Loss Percentage', barmode='group',
                     title="Penalties vs Win-Loss Percentage",
                     color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='',  
            coloraxis_colorbar=dict(title="Win-Loss %"),
            height=700
        )

    elif selected_metric == "Total Penalties Comparison":
        fig = px.bar(total_penalties, 
             x='Team', 
             y='Penalties', 
             title="Total Penalties Comparison",
             color='Penalties',
             color_continuous_scale='Bluered')
        
        fig.update_layout(
            xaxis_title='', 
            yaxis_title='Penalties',
            height=700,
            coloraxis_showscale=False
        )

    fig.show()
    display(metrics_dropdown_18)

metrics_dropdown_18.observe(lambda change: update_plot_18(change['new']), names='value')
display(metrics_dropdown_18)

Dropdown(description='Metric:', index=1, options=('Sum of Penalties & Success', 'Total Penalties Comparison'),…

In [447]:
metrics_dropdown_19 = widgets.Dropdown(
    options=["Sum of Penalty Yards & Success", "Total Penalty Yards Comparison"],
    description="Metric:"
)

NFL_stats_sorted = NFL_stats.sort_values(by='Win-Loss Percentage', ascending=False)

total_penalty_yards = NFL_stats.groupby('Team', as_index=False)['Penalty Yards'].sum()

total_penalty_yards = total_penalty_yards.sort_values(by='Penalty Yards', ascending=True)

def update_plot_19(selected_metric):
    clear_output(wait=True)

    if selected_metric == "Sum of Penalty Yards & Success":
        fig = px.bar(NFL_stats_sorted, x='Team', y='Penalty Yards', color='Win-Loss Percentage', barmode='group',
                     title="Penalty Yards vs Win-Loss Percentage",
                     color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='',  
            coloraxis_colorbar=dict(title="Win-Loss %"),
            height=700
        )

    elif selected_metric == "Total Penalty Yards Comparison":
        fig = px.bar(total_penalty_yards, 
             x='Team', 
             y='Penalty Yards', 
             title="Total Penalty Yards Comparison",
             color='Penalty Yards',
             color_continuous_scale='Bluered')
        
        fig.update_layout(
            xaxis_title='', 
            yaxis_title='Penalty Yards',
            height=700,
            coloraxis_showscale=False
        )

    fig.show()
    display(metrics_dropdown_19)

metrics_dropdown_19.observe(lambda change: update_plot_19(change['new']), names='value')
display(metrics_dropdown_19)

Dropdown(description='Metric:', index=1, options=('Sum of Penalty Yards & Success', 'Total Penalty Yards Compa…

In [448]:
metrics_dropdown_20 = widgets.Dropdown(
    options=["Penalty FD Given & Success", "Total Penalty FD Given"],
    description="Metric:"
)

NFL_stats_sorted = NFL_stats.sort_values(by='Win-Loss Percentage', ascending=False)

total_FD_given = NFL_stats.groupby('Team', as_index=False)['Penalty First Downs Given'].sum()

total_FD_given = total_FD_given.sort_values(by='Penalty First Downs Given', ascending=True)

def update_plot_20(selected_metric):
    clear_output(wait=True)

    if selected_metric == "Penalty FD Given & Success":
        fig = px.bar(NFL_stats_sorted, x='Team', y='Penalty First Downs Given', color='Win-Loss Percentage', barmode='group',
                     title="Automatic First Downs Allowed vs Win-Loss Percentage",
                     color_continuous_scale='Bluered_r')
        
        fig.update_layout(
            xaxis_title='',
            yaxis_title='Automatic First Downs Allowed',  
            coloraxis_colorbar=dict(title="Win-Loss %"),
            height=700
        )

    elif selected_metric == "Total Penalty FD Given":
        fig = px.bar(total_FD_given, 
             x='Team', 
             y='Penalty First Downs Given', 
             title="Total Automatic First Downs Allowed Comparison",
             color='Penalty First Downs Given',
             color_continuous_scale='Bluered')
        
        fig.update_layout(
            xaxis_title='', 
            yaxis_title='Automatic First Downs Allowed',
            height=700,
            coloraxis_showscale=False
        )

    fig.show()
    display(metrics_dropdown_20)

metrics_dropdown_20.observe(lambda change: update_plot_20(change['new']), names='value')
display(metrics_dropdown_20)

Dropdown(description='Metric:', index=1, options=('Penalty FD Given & Success', 'Total Penalty FD Given'), val…

In [481]:
team_dropdown_29 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metrics_dropdown_29 = widgets.Dropdown(
    options=["Penalties", "Penalty Yards"],
    description="Metric:"
)

def update_plot_29(selected_team, selected_metric):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]

    fig = go.Figure()

    margin_color = primary_team_colors.get(selected_team)

    if selected_metric == "Penalties":
        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Penalties'], 
                                 mode='lines', name='Penalties',
                                 line=dict(color=margin_color)))

        fig.update_layout(title=f"Penalties Over Time for the {selected_team}",
                          xaxis_title="Year",
                          yaxis_title="Penalties",
                          legend_title="")
        
        fig.update_yaxes(range=[54, 164])

    elif selected_metric == "Penalty Yards":
        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Penalty Yards'], 
                                 mode='lines', name='Penalty Yards',
                                 line=dict(color=margin_color)))

        fig.update_layout(title=f"Penalty Yards Over Time for the {selected_team}",
                          xaxis_title="Year",
                          yaxis_title="Penalty Yards",
                          legend_title="")
        
        fig.update_yaxes(range=[405, 1368])

    fig.show()
    display(team_dropdown_29)
    display(metrics_dropdown_29)

def on_dropdown_change(change):
    update_plot_29(team_dropdown_29.value, metrics_dropdown_29.value)

team_dropdown_29.observe(on_dropdown_change, names='value')
metrics_dropdown_29.observe(on_dropdown_change, names='value')

display(team_dropdown_29)
display(metrics_dropdown_29)

Dropdown(description='Team:', index=29, options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'N…

Dropdown(description='Metric:', index=1, options=('Penalties', 'Penalty Yards'), value='Penalty Yards')

In [449]:
team_dropdown_21 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metrics_dropdown_21 = widgets.Dropdown(
    options=["Teams' Discipline & Success", "Teams' Discipline Through Seasons"],
    description="Metric:"
)

def update_plot_21(selected_team, selected_metric):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]

    fig = go.Figure()

    if selected_metric == "Teams' Discipline & Success":

        hover_text = [
            f"Year: {year}<br>Win-Loss %: {win_loss_pct}" 
            for year, win_loss_pct in zip(filtered_data['Year'], filtered_data['Win-Loss Percentage'])
        ]

        fig.add_trace(go.Scatter(x=filtered_data['Penalties'], y=filtered_data['Penalty Yards'],
                                  mode='markers', hovertext=hover_text,
                                  marker=dict(color=filtered_data['Win-Loss Percentage'],
                                              colorscale='Bluered_r',
                                              colorbar=dict(title="Win/Loss %"),
                                              size=15)))
        
        fig.update_layout(title=f"{selected_team}' Discipline vs Success",
                          xaxis_title="Penalties",
                          yaxis_title="Yards")
        
    elif selected_metric == "Teams' Discipline Through Seasons":

        penalties_color = offensive_attempts_color_map.get(f"{selected_team}_Passing")
        yards_color = offensive_attempts_color_map.get(f"{selected_team}_Rushing")

        filtered_data['Normalized Penalties'] = (filtered_data['Penalties'] - filtered_data['Penalties'].min()) / (filtered_data['Penalties'].max() - filtered_data['Penalties'].min())
        filtered_data['Normalized Yards'] = (filtered_data['Penalty Yards'] - filtered_data['Penalty Yards'].min()) / (filtered_data['Penalty Yards'].max() - filtered_data['Penalty Yards'].min())
        
        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Normalized Penalties'], 
                                 fill='tozeroy', name='Penalties', mode='lines', line=dict(color=penalties_color)))

        fig.add_trace(go.Scatter(x=filtered_data['Year'], y=filtered_data['Normalized Yards'], 
                                 fill='tozeroy', name='Penalty Yards', mode='lines', line=dict(color=yards_color)))

        fig.update_layout(title=f'Normalized Penalties and Penalty Yards for the {selected_team}', 
                          xaxis_title='Year', yaxis_title='Normalized Value')

    
    fig.show()

    display(team_dropdown_21)
    display(metrics_dropdown_21)

def on_dropdown_change(change):
    update_plot_21(team_dropdown_21.value, metrics_dropdown_21.value)

team_dropdown_21.observe(on_dropdown_change, names='value')
metrics_dropdown_21.observe(on_dropdown_change, names='value')

display(team_dropdown_21)
display(metrics_dropdown_21)

Dropdown(description='Team:', options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'New York Je…

Dropdown(description='Metric:', options=("Teams' Discipline & Success", "Teams' Discipline Through Seasons"), …

In [450]:
team_dropdown_22 = widgets.Dropdown(
    options=NFL_stats['Team'].unique(), 
    description="Team:"
)

metric_dropdown_22 = widgets.Dropdown(
    options=["Penalties' Spread", "Penalty Yards' Spread"],  
    description="Metric:"
)

metric_mapping = {
    "Penalties' Spread": 'Penalties',
    "Penalty Yards' Spread": 'Penalty Yards'
}

def update_plot_22(selected_team, selected_metric):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Team'] == selected_team]

    y_column = metric_mapping.get(selected_metric)

    team_color = primary_team_colors.get(selected_team)

    if y_column in filtered_data.columns:

        fig = px.box(filtered_data, x='Team', y=y_column, 
                     title=f"{selected_metric} for the {selected_team}", labels={y_column: selected_metric.capitalize()},
                     color_discrete_sequence=[team_color])
    
        fig.update_layout(coloraxis_colorbar=dict(title=""), yaxis_title='', xaxis_title='',
                          showlegend=False, xaxis=dict(showticklabels=False))
        fig.show()
    
    display(team_dropdown_22)
    display(metric_dropdown_22)

def handle_change_22(change):
    update_plot_22(team_dropdown_22.value, metric_dropdown_22.value)

team_dropdown_22.observe(handle_change_22, names='value')
metric_dropdown_22.observe(handle_change_22, names='value')

display(team_dropdown_22)
display(metric_dropdown_22)

Dropdown(description='Team:', options=('New England Patriots', 'Miami Dolphins', 'Buffalo Bills', 'New York Je…

Dropdown(description='Metric:', index=1, options=("Penalties' Spread", "Penalty Yards' Spread"), value="Penalt…

In [482]:
year_dropdown_30 = widgets.Dropdown(
    options=NFL_stats['Year'].unique(), 
    description="Year:"
)

def update_plot_30(selected_year):
    clear_output(wait=True)
    filtered_data = NFL_stats[NFL_stats['Year'] == selected_year]

    fig = go.Figure()

    for team in filtered_data['Team'].unique():
        team_data = filtered_data[filtered_data['Team'] == team]
        team_color = real_primary_team_colors.get(team)
        team_record = team_data['Win-Loss Percentage'].values[0]
        
        fig.add_trace(go.Scatter(
            x=team_data['Penalties'], 
            y=team_data['Win-Loss Percentage'],
            mode='markers', 
            name=team,
            marker=dict(
                size=team_data['Win-Loss Percentage'] * 100,
                color=team_color
            )
        ))

    fig.update_layout(
        title=f"Penalties vs Win-Loss Percentage for Teams in {selected_year}",
        xaxis_title="Penalties",
        yaxis_title="Win-Loss Percentage",
        legend_title="Team",
        height=600
    )
    
    fig.show()
    display(year_dropdown_30)

def on_dropdown_change_30(change):
    update_plot_30(year_dropdown_30.value)

year_dropdown_30.observe(on_dropdown_change_30, names='value')

display(year_dropdown_30)

Dropdown(description='Year:', index=6, options=(2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 20…

In [484]:
super_bowl_winners = NFL_stats[NFL_stats['SuperBowl Win'] > 0]

sorted_winners = super_bowl_winners.sort_values(by=['SuperBowl Win', 'Penalties'], ascending=[False, False])

fig = px.bar(sorted_winners, x='Team', y='Penalties', color='Penalties', 
             title='Penalties of Super Bowl Winners', 
             color_continuous_scale='Bluered', hover_data={'Year': True})

fig.update_layout(yaxis_title='Penalties',
                  xaxis={'categoryorder':'total ascending'}, xaxis_title='',
                  height=600)

fig.show()

In [485]:
super_bowl_winners = NFL_stats[NFL_stats['SuperBowl Win'] > 0]

sorted_winners = super_bowl_winners.sort_values(by=['SuperBowl Win', 'Penalty Yards'], ascending=[False, False])

fig = px.bar(sorted_winners, x='Team', y='Penalty Yards', color='Penalty Yards', 
             title='Penalty Yards of Super Bowl Winners', 
             color_continuous_scale='Bluered', hover_data={'Year': True})

fig.update_layout(yaxis_title='Penalty Yards',
                  xaxis={'categoryorder':'total ascending'}, xaxis_title='',
                  height=600)

fig.show()

In [451]:
penalties_correlation_df = NFL_stats[['Penalties', 'Penalty Yards', 'Penalty First Downs Given', 'SuperBowl Win']]


penalties_correlation_matrix = penalties_correlation_df.corr()

fig = px.imshow(penalties_correlation_matrix, 
                text_auto=True, 
                color_continuous_scale='bluered_r', 
                zmin=-1, zmax=1, 
                labels={'color': 'Correlation'},
                title='Correlation Between Discipline and Championship Success')

fig.update_layout(height=600)

fig.show()

In [452]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Penalties']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Penalties']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in penalties between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in penalties between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 5939.0, P-Value = 0.30607743567983214, There is no significant difference in penalties between teams that won and teams that did not win the Super Bowl.


In [453]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Penalty Yards']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Penalty Yards']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in penalty yards between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in penalty yards between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 6876.5, P-Value = 0.9631084960643058, There is no significant difference in penalty yards between teams that won and teams that did not win the Super Bowl.


In [454]:
group_1 = NFL_stats[NFL_stats['SuperBowl Win'] == 1]['Penalty First Downs Given']
group_2 = NFL_stats[NFL_stats['SuperBowl Win'] == 0]['Penalty First Downs Given']

stat, p_value = mannwhitneyu(group_1, group_2)

if p_value > 0.05:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is no significant difference in automatic first downs allowed between teams that won and teams that did not win the Super Bowl.')
else:
  print(f'Mann-Whitney U Test: Stat = {stat}, P-Value = {p_value}, There is a significant difference in automatic first downs allowed between teams that won and teams that did not win the Super Bowl.')

Mann-Whitney U Test: Stat = 8181.5, P-Value = 0.12405613284175279, There is no significant difference in automatic first downs allowed between teams that won and teams that did not win the Super Bowl.


There is not a strong linear relationship between penalty discipline and Super Bowl wins, and neither a difference in the distribution of them.